<a href="https://colab.research.google.com/github/victgab20/polaridade-analise-utl_corpus/blob/main/Polaridade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import time
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from transformers import BertTokenizer


In [ ]:
!python --version


Python 3.11.11


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_test_apps = pd.read_pickle("/content/drive/MyDrive/dataset/UTLCorpus/test_apps.pkl")
dataset_train_apps = pd.read_pickle("/content/drive/MyDrive/dataset/UTLCorpus/train_apps.pkl")
dataset_dev_apps= pd.read_pickle("/content/drive/MyDrive/dataset/UTLCorpus/dev_apps.pkl")

dataset_test = pd.read_pickle("/content/drive/MyDrive/dataset/UTLCorpus/test_filmes.pkl")
dataset_train = pd.read_pickle("/content/drive/MyDrive/dataset/UTLCorpus/train_filmes.pkl")
dataset_dev = pd.read_pickle("/content/drive/MyDrive/dataset/UTLCorpus/dev_filmes.pkl")

In [ ]:
# Converter 'stars' para numérico e remover avaliações neutras (estrelas == 3)
def pre_processing(df: pd.DataFrame):
    df["stars"] = pd.to_numeric(df["stars"], errors="coerce")
    original_shape = df.shape[0]
    df = df[df["stars"] != 3]
    removed_rows = original_shape - df.shape[0]

    # Criar a variável alvo: 1 para positivo (stars > 3), 0 para negativo (stars < 3)
    df["label"] = (df["stars"] > 3).astype(int)

    # Selecionar apenas as colunas úteis
    y = df["label"]
    df = df[["text"]]

    # Preencher valores ausentes (se houver)
    df.fillna(0, inplace=True)
    return df, y


dataset_test_apps_processed, y_test_apps = pre_processing(dataset_test_apps)
dataset_train_apps_processed, y_train_apps = pre_processing(dataset_train_apps)
dataset_dev_apps_processed, y_dev_apps = pre_processing(dataset_dev_apps)
print(dataset_test_apps_processed.shape,y_test_apps.shape)
print(dataset_train_apps_processed.shape, y_train_apps.shape)
print(dataset_dev_apps_processed.shape, y_dev_apps.shape)

dataset_test_filmes_processed, y_test_filmes = pre_processing(dataset_test)
dataset_train_filmes_processed, y_train_filmes = pre_processing(dataset_train)
dataset_dev_filmes_processed, y_dev_filmes = pre_processing(dataset_dev)

print(dataset_test_filmes_processed.shape,y_test_filmes.shape)
print(dataset_train_filmes_processed.shape, y_train_filmes.shape)
print(dataset_dev_filmes_processed.shape, y_dev_filmes.shape)

<ipython-input-3-448e8b1f59ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = (df["stars"] > 3).astype(int)
<ipython-input-3-448e8b1f59ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = (df["stars"] > 3).astype(int)
<ipython-input-3-448e8b1f59ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

(175801, 1) (175801,)
(615281, 1) (615281,)
(87845, 1) (87845,)


<ipython-input-3-448e8b1f59ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = (df["stars"] > 3).astype(int)


(339473, 1) (339473,)
(1188408, 1) (1188408,)
(169857, 1) (169857,)


<ipython-input-3-448e8b1f59ba>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = (df["stars"] > 3).astype(int)


In [ ]:
y_dev_filmes.value_counts()

,count
label,
1,120722
0,49135


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Função para separar os dados em mini-batches
def generate_batches(X, y, batch_size=500):
    num_samples = len(X)

    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)

        # Extrair o mini-batch correspondente para X e y
        X_batch = X[start_idx:end_idx]
        y_batch = y[start_idx:end_idx]

        yield X_batch, y_batch

# Vetorização usando TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Defina o número máximo de características para controlar a memória

# Ajuste o vetorizar para os dados de treino (fit)
vectorizer.fit(dataset_train_apps_processed["text"])

# Defina o tamanho do batch
batch_size = 500

# Exemplo de como usar os mini-batches
for X_batch, y_batch in generate_batches(dataset_train_apps_processed["text"], y_train_apps, batch_size=batch_size):
    # Vetorizar o mini-batch de X (texto)
    X_batch_vectorized = vectorizer.transform(X_batch)


In [ ]:
class NaiveBayesNN(nn.Module):
    def __init__(self, input_dim):
        super(NaiveBayesNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)  # Uma camada linear
        self.sigmoid = nn.Sigmoid()  # Função de ativação para a classificação binária

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.fc = nn.Linear(input_dim, 1)  # Camada linear
        self.sigmoid = nn.Sigmoid()  # Função sigmoide para a saída binária

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from google.colab import drive

class TensorBoardLogger:
    def __init__(self, log_dir):
        # Inicializa o SummaryWriter
        self.writer = SummaryWriter(log_dir)

    def log_epoch(self, epoch, train_loss, train_accuracy, train_precision, train_recall, train_f1,
                  test_accuracy, test_precision, test_recall, test_f1):
        # Registra as métricas de treinamento
        self.writer.add_scalar('Loss/train', train_loss, epoch)
        self.writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        self.writer.add_scalar('Precision/train', train_precision, epoch)
        self.writer.add_scalar('Recall/train', train_recall, epoch)
        self.writer.add_scalar('F1 Score/train', train_f1, epoch)

        # Registra as métricas de teste
        self.writer.add_scalar('Accuracy/test', test_accuracy, epoch)
        self.writer.add_scalar('Precision/test', test_precision, epoch)
        self.writer.add_scalar('Recall/test', test_recall, epoch)
        self.writer.add_scalar('F1 Score/test', test_f1, epoch)

    def close(self):
        # Fecha o writer
        self.writer.close()

In [ ]:
# Definir parâmetros
input_dim = X_batch_vectorized.shape[1]  # número de features após a vetorização
batch_size = 500  # Defina o tamanho do batch
learning_rate = 0.001
epochs = 5  # Número de épocas para o treinamento

# Criar o modelo
model = NaiveBayesNN(input_dim=input_dim)

# Função de perda e otimizador
criterion = nn.BCELoss()  # Usando a Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Colocar o modelo em modo de treinamento
model.train()


NaiveBayesNN(
  (fc1): Linear(in_features=5000, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

,text
0,"Gente, isso é uma comédia romântica. Todo mund..."
1,"A partir de setembro de 1943, socialistas, ana..."
2,O melhor filme do mundo <3
3,Incrível. É bom demais ver que ainda existem r...
4,A cena da sacola.
...,...
1286012,"Caraaamba, eu amei esse filme.Mesmo não sendo ..."
1286013,Não sei se só eu não entendi a parte em que o ...
1286014,"Nossa esse filme é muito legal,to doido pra co..."
1286015,Divino!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score

# Defina o tamanho do batch
batch_size = 1500

log_dir = '/content/drive/MyDrive/runs/your_experiment_name/NaiveBayesNN_apps'  # Altere o nome do diretório conforme necessário
logger = TensorBoardLogger(log_dir)

# Vetorização usando TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Defina o número máximo de características para controlar a memória

# Ajuste o vetorizar para os dados de treino (fit)
vectorizer.fit(dataset_train_apps_processed["text"])

# Função de treinamento com avaliação em cada época
epochs = 15
for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0
    all_preds = []  # Lista para armazenar previsões
    all_labels = []  # Lista para armazenar rótulos verdadeiros

    # Gerar batches e treinar
    for X_batch, y_batch in generate_batches(dataset_train_apps_processed["text"], y_train_apps, batch_size=batch_size):
        # Vetorizar o batch de X (texto)
        X_batch_vectorized = vectorizer.transform(X_batch).toarray()  # Conversão para numpy array
        X_batch_tensor = torch.tensor(X_batch_vectorized, dtype=torch.float32)  # Converter para tensor

        # Converter y_batch para tensor
        y_batch_tensor = torch.tensor(y_batch.values, dtype=torch.float32).view(-1, 1)

        # Passar os dados pelo modelo
        optimizer.zero_grad()  # Zerando os gradientes do otimizador
        outputs = model(X_batch_tensor)  # Obter as saídas do modelo

        # Calcular a perda
        loss = criterion(outputs, y_batch_tensor)
        total_loss += loss.item()

        # Calcular a acurácia
        predicted = (outputs >= 0.5).float()
        total_correct += (predicted == y_batch_tensor).sum().item()
        total_samples += y_batch_tensor.size(0)

        # Armazenar previsões e rótulos para métricas
        all_preds.extend(predicted.numpy())
        all_labels.extend(y_batch_tensor.numpy())

        # Backpropagation e otimização
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / (total_samples / batch_size)  # Média da perda por batch
    accuracy = total_correct / total_samples

    # Calcular métricas adicionais
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)  # A variável f1 foi renomeada corretamente

    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}, '
          f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # Avaliação no conjunto de teste após cada época
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():  # Desativar o cálculo dos gradientes durante a avaliação
        total_correct_test = 0
        total_samples_test = 0
        all_preds_test = []  # Lista para previsões no teste
        all_labels_test = []  # Lista para rótulos verdadeiros no teste

        for X_batch, y_batch in generate_batches(dataset_test_apps_processed["text"], y_test_apps, batch_size=batch_size):
            # Vetorizar o batch de X (texto)
            X_batch_vectorized = vectorizer.transform(X_batch).toarray()  # Conversão para numpy array
            X_batch_tensor = torch.tensor(X_batch_vectorized, dtype=torch.float32)  # Converter para tensor

            # Converter y_batch para tensor
            y_batch_tensor = torch.tensor(y_batch.values, dtype=torch.float32).view(-1, 1)

            # Passar os dados pelo modelo
            outputs = model(X_batch_tensor)  # Obter as saídas do modelo

            # Calcular a acurácia
            predicted = (outputs >= 0.5).float()
            total_correct_test += (predicted == y_batch_tensor).sum().item()
            total_samples_test += y_batch_tensor.size(0)

            # Armazenar previsões e rótulos para métricas no teste
            all_preds_test.extend(predicted.numpy())
            all_labels_test.extend(y_batch_tensor.numpy())

        # Calcular métricas de teste
        test_accuracy = total_correct_test / total_samples_test
        test_precision = precision_score(all_labels_test, all_preds_test)
        test_recall = recall_score(all_labels_test, all_preds_test)
        test_f1 = f1_score(all_labels_test, all_preds_test)

        print(f'Test Accuracy after epoch {epoch + 1}: {test_accuracy:.4f}, '
              f'Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1 Score: {test_f1:.4f}')

        # Corrigido para passar a variável f1
        logger.log_epoch(epoch, avg_loss, accuracy, precision, recall, f1,
                         test_accuracy, test_precision, test_recall, test_f1)
        torch.save(model.state_dict(), "polarity_apps.pt")

logger.close()


Epoch [1/15], Loss: 0.5643, Accuracy: 0.7874, Precision: 0.7852, Recall: 0.9980, F1 Score: 0.8789
Test Accuracy after epoch 1: 0.8048, Precision: 0.7996, Recall: 0.9969, F1 Score: 0.8874
Epoch [2/15], Loss: 0.4333, Accuracy: 0.8347, Precision: 0.8283, Recall: 0.9917, F1 Score: 0.9027
Test Accuracy after epoch 2: 0.8488, Precision: 0.8427, Recall: 0.9886, F1 Score: 0.9099
Epoch [3/15], Loss: 0.3657, Accuracy: 0.8668, Precision: 0.8636, Recall: 0.9829, F1 Score: 0.9194
Test Accuracy after epoch 3: 0.8712, Precision: 0.8697, Recall: 0.9800, F1 Score: 0.9216
Epoch [4/15], Loss: 0.3247, Accuracy: 0.8827, Precision: 0.8843, Recall: 0.9760, F1 Score: 0.9279
Test Accuracy after epoch 4: 0.8833, Precision: 0.8862, Recall: 0.9739, F1 Score: 0.9280
Epoch [5/15], Loss: 0.2979, Accuracy: 0.8914, Precision: 0.8969, Recall: 0.9711, F1 Score: 0.9325
Test Accuracy after epoch 5: 0.8906, Precision: 0.8969, Recall: 0.9697, F1 Score: 0.9319
Epoch [6/15], Loss: 0.2791, Accuracy: 0.8971, Precision: 0.9053, 

In [ ]:
# Função de treinamento com avaliação em cada época
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    total_correct = 0
    total_samples = 0

    # Gerar batches e treinar
    for X_batch, y_batch in generate_batches(dataset_train_apps_processed["text"], y_train_apps, batch_size=batch_size):
        # Vetorizar o batch de X (texto)
        X_batch_vectorized = vectorizer.transform(X_batch).toarray()  # Conversão para numpy array
        X_batch_tensor = torch.tensor(X_batch_vectorized, dtype=torch.float32)  # Converter para tensor

        # Converter y_batch para tensor
        y_batch_tensor = torch.tensor(y_batch.values, dtype=torch.float32).view(-1, 1)

        # Passar os dados pelo modelo
        optimizer.zero_grad()  # Zerando os gradientes do otimizador
        outputs = model(X_batch_tensor)  # Obter as saídas do modelo

        # Calcular a perda
        loss = criterion(outputs, y_batch_tensor)
        total_loss += loss.item()

        # Calcular a acurácia
        predicted = (outputs >= 0.5).float()
        total_correct += (predicted == y_batch_tensor).sum().item()
        total_samples += y_batch_tensor.size(0)

        # Backpropagation e otimização
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataset_train_apps_processed)
    accuracy = total_correct / total_samples
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

    # Avaliação no conjunto de teste após cada época
    model.eval()  # Colocar o modelo em modo de avaliação
    with torch.no_grad():  # Desativar o cálculo dos gradientes durante a avaliação
        total_correct_test = 0
        total_samples_test = 0

        for X_batch, y_batch in generate_batches(dataset_test_apps_processed["text"], y_test_apps, batch_size=batch_size):
            # Vetorizar o batch de X (texto)
            X_batch_vectorized = vectorizer.transform(X_batch).toarray()  # Conversão para numpy array
            X_batch_tensor = torch.tensor(X_batch_vectorized, dtype=torch.float32)  # Converter para tensor

            # Converter y_batch para tensor
            y_batch_tensor = torch.tensor(y_batch.values, dtype=torch.float32).view(-1, 1)

            # Passar os dados pelo modelo
            outputs = model(X_batch_tensor)  # Obter as saídas do modelo

            # Calcular a acurácia
            predicted = (outputs >= 0.5).float()
            total_correct_test += (predicted == y_batch_tensor).sum().item()
            total_samples_test += y_batch_tensor.size(0)

    test_accuracy = total_correct_test / total_samples_test
    print(f'Test Accuracy after epoch {epoch + 1}: {test_accuracy:.4f}')


Epoch [1/20], Loss: 0.0004, Accuracy: 0.9230
Test Accuracy after epoch 1: 0.9187
Epoch [2/20], Loss: 0.0004, Accuracy: 0.9231
Test Accuracy after epoch 2: 0.9186
Epoch [3/20], Loss: 0.0004, Accuracy: 0.9232
Test Accuracy after epoch 3: 0.9187


KeyboardInterrupt: 

In [ ]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.fc = nn.Linear(input_dim, 1)  # Camada linear
        self.sigmoid = nn.Sigmoid()  # Função sigmoide para a saída binária

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x